In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ${catalog_prefix}ridb.gold;

CREATE OR REPLACE TABLE ${catalog_prefix}ridb.gold.date_dimension (
  date_key DATE PRIMARY KEY,
  year INT,
  quarter INT,
  quarter_name STRING,
  month INT,
  month_name STRING,
  day INT,
  day_of_week INT,
  day_of_week_name STRING,
  week_of_year INT,
  is_weekend BOOLEAN
);

INSERT INTO ${catalog_prefix}ridb.gold.date_dimension
SELECT
  d AS date_key,
  year(d) AS year,
  quarter(d) AS quarter,
  CASE quarter(d)
    WHEN 1 THEN 'Q1'
    WHEN 2 THEN 'Q2'
    WHEN 3 THEN 'Q3'
    WHEN 4 THEN 'Q4'
  END AS quarter_name,
  month(d) AS month,
  date_format(d, 'MMMM') AS month_name,
  day(d) AS day,
  dayofweek(d) AS day_of_week,
  date_format(d, 'EEEE') AS day_of_week_name,
  weekofyear(d) AS week_of_year,
  CASE WHEN dayofweek(d) IN (7, 1) THEN TRUE ELSE FALSE END AS is_weekend
FROM (
  SELECT sequence(
    date_add(current_date(), -3652),  -- approx 10 years ago
    date_add(current_date(), 3652),   -- approx 10 years ahead
    INTERVAL 1 DAY
  ) AS date_seq
) s
LATERAL VIEW explode(date_seq) t AS d;